# Ocena razpoloženja (sentimenta)

Sentimentno analizo bomo opravili v računalniškem jeziku R, vendar ob podpori modela s transformersko arhitekturo, ki ga prikličemo iz Pythonovega okolja.

## Priprava Pythonovega virtualnega okolja

V virtualnem operacijskem sistemu Linux Googlovega kolaboratorija je treba najprej pripraviti Pythonovo virtualno okolje in namestiti Pythonove module, na katere se lahko sklicujejo funkcije v računalniškem jeziku R.

V nevirtualnih operacijskih sistemih je priprava preprostejša.

In [1]:
# If needed, run shell commands in R via system()
system("which python")
system("python --version")
system("sudo apt-get update -y")
system("sudo apt-get install -y python3-dev")
system("sudo apt-get install -y python3-venv")


## R knjižnica reticulate

Knjižnica reticulate je vmesnik za delo s Pythonom. Knjižnico namestimo z ukazom install.packages(), ki je značilen za R-ovo sinttakso. Prikličemo jo pa s funkcijo library().

In [2]:
install.packages("reticulate")
library(reticulate)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Ustvarjanje Pythonovega okolja za R

In [3]:
# virtualenv_create("bertopic", python = "/usr/local/bin/python")
system("python3 -m venv /root/.virtualenvs/bertopic")



In [4]:
system("pip install virtualenv")
system("virtualenv /root/.virtualenvs/bertopic")


## Namestitev Pythonovih knjižnic

Najprej s funkcijo use_virtualenv() aktiviramo zgoraj pripravljeno Pythonovo okolje "bertopic". Potem s py_install() nameščamo vse potrebne Pythonove knjižnice (module).

In [5]:
library(reticulate)
use_virtualenv("bertopic", required = TRUE)
py_install("pandas", envname = "bertopic")
py_install(c("numpy", "matplotlib"), envname = "bertopic")
py_install("transformers", envname = "bertopic")
py_install("python-docx", envname = "bertopic")


Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user pandas



Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user numpy matplotlib



Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user transformers



Using virtual environment 'bertopic' ...


+ /root/.virtualenvs/bertopic/bin/python -m pip install --upgrade --no-user python-docx



In [6]:
system("pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
system("pip install transformers")
system("pip install sentencepiece")
system("pip install protobuf")

## Priklic Pythonovih knjižnic

In [7]:
# Import necessary Python modules
# Inicializacija knjižnic
re <- import("re")
datetime <- import("datetime")
docx <- import("docx")
Document <- docx$Document
pd <- import("pandas")
np <- import("numpy")
transformers <- import("transformers")
pipeline <- transformers$pipeline


## Namestitev knjižnic R

In [8]:
install.packages(c("tidyverse", "tidytext"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



## Priklic knjižnic R

In [9]:
library(tidyverse)
library(tidytext)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Branje besedila

In [10]:
input_folder <- "/content/"
input_file <- "Sporocilo sodelavcem univerze o kibernapadu.txt"
text <- read_lines(file.path(input_folder, input_file), locale = locale(encoding = 'UTF-8'))
cat(head(text, 5), sep = "\n")

Spoštovane sodelavke in sodelavci!
 
23.10.2024 je bil v večernih urah izveden kibernetski napad z izsiljevalsko programsko kodo na vso informacijsko infrastrukturo Univerze v Mariboru, s čimer so bile onemogočene vse storitve, ki so nam tekoče na voljo, kot so: elektronska pošta, okolje spletne učilnice, spletne strani, digitalna identiteta, uporaba akademskega, kadrovskega in finančnega informacijskega sistema, uporaba orodja MS-Teams, storitev OneDrive in ostale. Natančnega vektorja napada kljub podrobni forenzični analizi neodvisnih forenzičnih strokovnjakov v tem trenutku še ni mogoče določiti, pri čemer pa je znan najverjetnejši prvo napadeni strežnik, na katerem je napadalec pustil tudi sporočilo o komunikacijskem kanalu, preko katerega ga lahko kontaktiramo. Kanal ni bil uporabljen, Univerza v Mariboru napadalca po tem kanalu, ki je zahteval namestitev posebne programske opreme, ni kontaktirala. Pri tem je pomembno poudariti, da vzrok napada ni bila menjava požarnega zidu in da

## Branje nezaželenih besed

Če potrebno, bomo izločili funkcijske besede, ki ne prispevajo k tematiki.

In [11]:
input_folder <- "/content/"
input_file <- "all_stopwords.txt"
all_stopwords <- read_lines(file.path(input_folder, input_file))

## Razdelitev na povedi

Naš model najbolje deluje s povedmi. Zato razdelimo besedilo na povedi, bistveno merilo za segmentacijo pa so končna ločila.

In [12]:
txt <- tibble(text = text)
sents <- txt %>%
  unnest_sentences(sentences, text, to_lower = FALSE)
sentences <- str_squish(sents$sentences)
head(sentences)

[1] "Spoštovane sodelavke in sodelavci!"                                                                                                                                                                                                                                                                                                                                                                                                               
[2] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                 
[3] "23.10.2024 je bil v večernih urah izveden kibernetski napad z izsiljevalsko programsko kodo na vso informacijsko infrastrukturo Univerze v Mariboru, s čimer so bile onemogočene vse storitve, ki so nam tekoče na voljo, kot so: elektronska pošta, okolje spletne učilnice, spletne strani, digitalna identiteta, uporaba akademskega, kadrovskega in finančnega informacijskega sistema, uporaba orodja MS-Teams, storitev OneDrive in ostale."
[4] "Natančnega vektorja napada kljub podrobni forenzični analizi neodvisnih forenzičnih strokovnjakov v tem trenutku še ni mogoče določiti, pri čemer pa je znan najverjetnejši prvo napadeni strežnik, na katerem je napadalec pustil tudi sporočilo o komunikacijskem kanalu, preko katerega ga lahko kontaktiramo."                                                                                                                                
[5] "Kanal ni bil uporabljen, Univerza v Mariboru napadalca po tem kanalu, ki je zahteval namestitev posebne programske opreme, ni kontaktirala."                                                                                                                                                                                                                                                                                                      
[6] "Pri tem je pomembno poudariti, da vzrok napada ni bila menjava požarnega zidu in da smo imeli na UM ustrezno in po pravilih urejen sistem varovanja naše informacijsko-komunikacijske infrastrukture in sistem varovanih kopij podatkov."

## Priklic modela za sentimentno analizo

In [15]:
# Naloži model sentimentne analize
sentiment_analysis <- pipeline('sentiment-analysis', model = 'cardiffnlp/twitter-xlm-roberta-base-sentiment')


## Analiza razpoloženja (sentimenta)

In [16]:
# Analiziraj sentiment
results <- lapply(sentences, function(sentence) {
  # sentiment <- py_to_r(sentiment_analysis(sentence))[[1]]
  sentiment <- sentiment_analysis(sentence)[[1]]
  list(sentence = sentence, label = sentiment$label, score = sentiment$score)
})



In [20]:
head(results)

[[1]]
[[1]]$sentence
[1] "Spoštovane sodelavke in sodelavci!"

[[1]]$label
[1] "negative"

[[1]]$score
[1] 0.4459


[[2]]
[[2]]$sentence
[1] ""

[[2]]$label
[1] "negative"

[[2]]$score
[1] 0.3472995


[[3]]
[[3]]$sentence
[1] "23.10.2024 je bil v večernih urah izveden kibernetski napad z izsiljevalsko programsko kodo na vso informacijsko infrastrukturo Univerze v Mariboru, s čimer so bile onemogočene vse storitve, ki so nam tekoče na voljo, kot so: elektronska pošta, okolje spletne učilnice, spletne strani, digitalna identiteta, uporaba akademskega, kadrovskega in finančnega informacijskega sistema, uporaba orodja MS-Teams, storitev OneDrive in ostale."

[[3]]$label
[1] "neutral"

[[3]]$score
[1] 0.8377048


[[4]]
[[4]]$sentence
[1] "Natančnega vektorja napada kljub podrobni forenzični analizi neodvisnih forenzičnih strokovnjakov v tem trenutku še ni mogoče določiti, pri čemer pa je znan najverjetnejši prvo napadeni strežnik, na katerem je napadalec pustil tudi sporočilo o komunikacijskem kanalu, preko katerega ga lahko kontaktiramo."

[[4]]$label
[1] "neutral"

[[4]]$score
[1] 0.6118678


[[5]]
[[5]]$sentence
[1] "Kanal ni bil uporabljen, Univerza v Mariboru napadalca po tem kanalu, ki je zahteval namestitev posebne programske opreme, ni kontaktirala."

[[5]]$label
[1] "neutral"

[[5]]$score
[1] 0.5066735


[[6]]
[[6]]$sentence
[1] "Pri tem je pomembno poudariti, da vzrok napada ni bila menjava požarnega zidu in da smo imeli na UM ustrezno in po pravilih urejen sistem varovanja naše informacijsko-komunikacijske infrastrukture in sistem varovanih kopij podatkov."

[[6]]$label
[1] "neutral"

[[6]]$score
[1] 0.6493952

## Pretvorba v podatkovni niz

In [26]:
# Convert results to a data.frame
results_df <- do.call(rbind, lapply(results, as.data.frame))
rownames(results_df) <- NULL  # Remove row names
head(results_df)


,sentence,label,score
,<chr>,<chr>,<dbl>
1,Spoštovane sodelavke in sodelavci!,negative,0.4459000
2,,negative,0.3472995
3,"23.10.2024 je bil v večernih urah izveden kibernetski napad z izsiljevalsko programsko kodo na vso informacijsko infrastrukturo Univerze v Mariboru, s čimer so bile onemogočene vse storitve, ki so nam tekoče na voljo, kot so: elektronska pošta, okolje spletne učilnice, spletne strani, digitalna identiteta, uporaba akademskega, kadrovskega in finančnega informacijskega sistema, uporaba orodja MS-Teams, storitev OneDrive in ostale.",neutral,0.8377048
4,"Natančnega vektorja napada kljub podrobni forenzični analizi neodvisnih forenzičnih strokovnjakov v tem trenutku še ni mogoče določiti, pri čemer pa je znan najverjetnejši prvo napadeni strežnik, na katerem je napadalec pustil tudi sporočilo o komunikacijskem kanalu, preko katerega ga lahko kontaktiramo.",neutral,0.6118678
5,"Kanal ni bil uporabljen, Univerza v Mariboru napadalca po tem kanalu, ki je zahteval namestitev posebne programske opreme, ni kontaktirala.",neutral,0.5066735
6,"Pri tem je pomembno poudariti, da vzrok napada ni bila menjava požarnega zidu in da smo imeli na UM ustrezno in po pravilih urejen sistem varovanja naše informacijsko-komunikacijske infrastrukture in sistem varovanih kopij podatkov.",neutral,0.6493952


In [38]:
write.csv(results_df, file = "results.csv", row.names = FALSE)


## Dodeljevanje barv

In [21]:
# Map sentiment labels to colors
get_color <- function(label) {
  if (label == "POSITIVE") {
    return("green")
  } else if (label == "NEGATIVE") {
    return("red")
  } else {
    return("gray")
  }
}


## Predloga spletne strani

In [22]:
# Ustvari HTML vsebino
html_content <- "<!DOCTYPE html>
<html>
<head>
    <title>Sentiment Analysis</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.8; margin: 20px; }
        .positive { color: green; }
        .neutral { color: gray; }
        .negative { color: red; }
    </style>
</head>
<body>
<h1>Sentiment Analysis Results</h1>
<p>Below are the sentences from the document, colored based on their sentiment:</p>
"



## Dodeljevanje povedi barvam

In [23]:
# Dodaj povedi v HTML vsebino
for (result in results) {
  color_class <- tolower(result$label)
  html_content <- paste0(html_content, "<p class='", color_class, "'>", result$sentence, "</p>\n")
}


In [24]:
# Zaključi HTML vsebino
html_content <- paste0(html_content, "</body></html>")


## Shrani HTML datoteko

In [25]:
# Shrani HTML datoteko
output_file <- "sentiment_analysis_colored.html"
writeLines(html_content, output_file)

cat("HTML file saved as", output_file, ". Open it in a browser to view the results.\n")


HTML file saved as sentiment_analysis_colored.html . Open it in a browser to view the results.
